In [1]:
import db_connect
import pandas

In [2]:
# 建立與MySQL的連線
my_connt_obj = db_connect.mysql_connect()
conn = my_connt_obj.connect()

In [3]:
# 取得三大法人期選的資料
df = pandas.read_sql("SELECT * FROM LegalDailyFutureOption WHERE TradeDate >'2021-01-01'", con=conn)

In [4]:
# 關閉連線
conn.close() 

In [5]:
# 觀察資料
df.tail()

,ID,TradeDate,TradeGroup,FutureLongQty,OptionLongQty,FutureLongAmount,OptionLongAmount,FutureShortQty,OptionShortQty,FutureShortAmount,...,OptionLongOIAmount,FutureShortOIQty,OptionShortOIQty,FutureShortOIAmount,OptionShortOIAmount,FutureOINetQty,OptionOINetQty,FutureOINetAmount,OptionOINetAmount,CreateTime
517,8135,2021-09-22,投信,5,8,16894,31,1967,7,6646254,...,1628,20998,0,63159162,0,-2517,59,-13424707,1628,2021-09-22 23:39:16
518,8136,2021-09-22,外資及陸資,270724,182544,354024668,700634,281646,185229,367253477,...,470997,324587,52994,210336008,554387,-250865,-24101,-125257885,-83390,2021-09-22 23:39:16
519,8137,2021-09-23,自營商,29090,159082,38781895,581873,30088,146242,39509192,...,707834,67046,91242,37291727,578932,31427,19799,53036513,128902,2021-09-23 23:44:47
520,8138,2021-09-23,投信,481,0,1642749,0,49,0,147945,...,1230,20698,0,62784800,0,-2085,59,-12074796,1230,2021-09-23 23:44:47
521,8139,2021-09-23,外資及陸資,239876,68623,290299165,384233,240279,70868,284570261,...,407816,332087,60947,215040049,514456,-251346,-26346,-120727849,-106640,2021-09-23 23:44:47


# 自營商

製作自營商的：期貨留倉為多單還是空單、留倉數量、變動比例。

In [25]:
# 自營 資料
df_investment = df.loc[df['TradeGroup'] == "自營商"]
df_investment.tail()

,ID,TradeDate,TradeGroup,FutureLongQty,OptionLongQty,FutureLongAmount,OptionLongAmount,FutureShortQty,OptionShortQty,FutureShortAmount,...,OptionLongOIAmount,FutureShortOIQty,OptionShortOIQty,FutureShortOIAmount,OptionShortOIAmount,FutureOINetQty,OptionOINetQty,FutureOINetAmount,OptionOINetAmount,CreateTime
507,8122,2021-09-13,自營商,54321,168418,63391474,397875,52103,180778,60094966,...,990959,96482,139212,55638077,784371,12324,14844,39056279,206588,2021-09-15 22:15:13
510,8125,2021-09-14,自營商,54880,220129,61894466,448193,53526,199685,65691737,...,1041502,97017,141527,55924249,787847,13659,35255,35373400,253655,2021-09-15 22:15:13
513,8131,2021-09-16,自營商,22897,141219,34517160,458717,24348,147532,32594020,...,556122,58559,90646,32132150,473098,34285,2232,51029866,83024,2021-09-16 22:59:03
516,8134,2021-09-22,自營商,41764,335919,66502081,888346,40626,385380,58454751,...,694084,64794,80397,35435315,633771,32789,6937,53292208,60313,2021-09-22 23:39:16
519,8137,2021-09-23,自營商,29090,159082,38781895,581873,30088,146242,39509192,...,707834,67046,91242,37291727,578932,31427,19799,53036513,128902,2021-09-23 23:44:47


In [26]:
# 取得不包含最後一筆之再前五筆資料
last_5_investment = df_investment["FutureOINetQty"][-6:-1]
# 取得最後一筆資料
last_investment = df_investment["FutureOINetQty"][-1:]
print(last_5_investment)
print(last_investment)

504    35537
507    12324
510    13659
513    34285
516    32789
Name: FutureOINetQty, dtype: int64
519    31427
Name: FutureOINetQty, dtype: int64


In [27]:
# signal_3_self: 自營商期貨留倉是否空單還是多單
# 最近一次交易日與前一次做比較
# True:多單；False：空單

signal_3_self = True if last_investment.iloc[0] > 0 else False
signal_3_self

True

In [28]:
# signal_4_self: 自營商期貨留倉數量是否增加
# 最近一次交易日與前一次的期貨留倉數量做比較
# True:增加；False：減少

signal_4_self = True if last_investment.iloc[0] >= last_5_investment.iloc[-1] else False
signal_4_self


False

In [30]:
# signal_5_self: 自營商期貨留倉數量變化多少
# 最近一次交易日與前一次的期貨留倉數量做相減
# 顯示為變化數量

signal_5_self = last_investment.iloc[0] - last_5_investment.iloc[-1]
signal_5_self


-1362

In [31]:
# signal_6_self: 自營商期貨留倉數變動比例
# 為小數顯示

signal_6_self = abs((last_investment.iloc[0] - last_5_investment.iloc[-1])/last_5_investment.iloc[-1])
signal_6_self

-0.0415383207783098

# 投信

製作投信的：期貨留倉為多單還是空單、留倉數量、變動比例。

In [32]:
# 投信 資料
df_investment = df.loc[df['TradeGroup'] == "投信"]
df_investment.tail()

,ID,TradeDate,TradeGroup,FutureLongQty,OptionLongQty,FutureLongAmount,OptionLongAmount,FutureShortQty,OptionShortQty,FutureShortAmount,...,OptionLongOIAmount,FutureShortOIQty,OptionShortOIQty,FutureShortOIAmount,OptionShortOIAmount,FutureOINetQty,OptionOINetQty,FutureOINetAmount,OptionOINetAmount,CreateTime
508,8123,2021-09-13,投信,19008,1,61503858,13,19147,2,61952468,...,22,19997,0,61233189,0,514,61,-2866446,22,2021-09-15 22:15:13
511,8126,2021-09-14,投信,18842,1,54056912,11,18623,1,53630444,...,27,19887,0,61188652,0,732,61,-2457964,27,2021-09-15 22:15:13
514,8132,2021-09-16,投信,4,0,13800,0,317,0,1066713,...,1024,20320,0,62119080,0,-629,58,-7197187,1024,2021-09-16 22:59:03
517,8135,2021-09-22,投信,5,8,16894,31,1967,7,6646254,...,1628,20998,0,63159162,0,-2517,59,-13424707,1628,2021-09-22 23:39:16
520,8138,2021-09-23,投信,481,0,1642749,0,49,0,147945,...,1230,20698,0,62784800,0,-2085,59,-12074796,1230,2021-09-23 23:44:47


In [33]:
# 取得不包含最後一筆之再前五筆資料
last_5_investment = df_investment["FutureOINetQty"][-6:-1]
# 取得最後一筆資料
last_investment = df_investment["FutureOINetQty"][-1:]

print(last_5_investment)
print(last_investment)

505    -316
508     514
511     732
514    -629
517   -2517
Name: FutureOINetQty, dtype: int64
520   -2085
Name: FutureOINetQty, dtype: int64


In [34]:
# signal_3_investment: 投信期貨留倉是否空單還是多單
# 最近一次交易日與前一次做比較
# True:多單；False：空單

signal_3_investment = True if last_investment.iloc[0] > 0 else False
signal_3_investment

False

In [35]:
# signal_4_investment: 投信期貨留倉數量是否增加
# 最近一次交易日與前一次的期貨留倉數量做比較
# True:增加；False：減少

signal_4_investment = True if last_investment.iloc[0] >= last_5_investment.iloc[-1] else False
signal_4_investment

True

In [36]:
# signal_5_investment: 投信期貨留倉數量變化多少
# 最近一次交易日與前一次的期貨留倉數量做相減
# 顯示為變化數量

signal_5_investment = last_investment.iloc[0] - last_5_investment.iloc[-1]
signal_5_investment

432

In [38]:
# signal_6_investment: 投信期貨留倉數變動比例
# 為小數顯示

signal_6_investment = abs((last_investment.iloc[0] - last_5_investment.iloc[-1])/last_5_investment.iloc[-1])
signal_6_investment

0.17163289630512515